# Groupby
<br><img align="left" src="http://drive.google.com/uc?export=view&id=17lLj-fLLYk6Dxcz7yBIX7bMEl4PAESBB" width=800 height=600>

- 하나의 데이터프레임에서 key column을 범주로 데이터들을 집계하고 싶을 때
```python
# Series 형식으로 뽑고 싶을 때
data.groupby(key_column)[value_column].function()
# 데이터프레임 형식으로 뽑고 싶을 때
data.groupby(key_column)[[value_column]].function()
```

In [12]:
import pandas as pd
import numpy as np
import random

# 난수고정
random.seed(777)

# 데이터프레임 생성
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.255730,-1.097117
1,a,two,0.263433,0.265841
2,b,one,0.963786,-0.239807
3,b,two,0.007764,-0.077920
4,a,one,0.859010,-0.051924


In [13]:
# key column을 기준으로, 나머지 모든 열들을 집계하고 싶으면 value_column을 지정해주지 않아도 된다.
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.288904,-0.294400
b,0.485775,-0.158864


In [14]:
# 두 개 이상의 key column을 지정해줄 수도 있다.
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.301640 -0.574521
     two   0.263433  0.265841
b    one   0.963786 -0.239807
     two   0.007764 -0.077920

In [15]:
# groupby() 안에 as_index = False로 넣으면 index값을 넣어줄 수 있다.
df.groupby(['key1', 'key2'], as_index = False).mean()

,key1,key2,data1,data2
0,a,one,0.301640,-0.574521
1,a,two,0.263433,0.265841
2,b,one,0.963786,-0.239807
3,b,two,0.007764,-0.077920


In [16]:
# value_column을 지정할수도, 여러개 지정할 수도 있다.
df.groupby(['key1', 'key2'])['data1'].mean()

key1  key2
a     one     0.301640
      two     0.263433
b     one     0.963786
      two     0.007764
Name: data1, dtype: float64

In [17]:
# 파이썬은 똑똑해서 value_column을 []로만 묶어줘도 DataFrame형식으로 return한다.
# 하지만 우리가 읽기 편하도록 [[]] 으로 묶어주는 연습을 하자.
df.groupby(['key1', 'key2'])[['data1','data2']].mean()

data1     data2
key1 key2                    
a    one   0.301640 -0.574521
     two   0.263433  0.265841
b    one   0.963786 -0.239807
     two   0.007764 -0.077920

In [26]:
# 예제. 다음과 같은 성적 파일에서 아래 항목을 구하시오.
#1. 각 학생의 평균 성적을 구하시오.

source.groupby(['이름'])['성적'].mean()
#2. 성별별 평균 성적을 구하시오.

source.groupby(['성별'])['성적'].mean()
#3. 각 과목의 평균 성적을 구하시오.
source.groupby(['과목'])['성적'].mean()

source = pd.read_csv('score.csv');source.head()

,이름,성별,과목,성적
0,갑,남,국,70
1,갑,남,수,60
2,갑,남,영,70
3,갑,남,탐1,80
4,갑,남,탐2,70


### aggregate function

- 집계함수는 기본적인 통계연산을 지원한다.

 `mean(), std(), sum(), count(), max(), min(), size() 등등`

- 직접 만든 함수를 사용하고 싶다면, agg() 메서드를 사용한다.
```python
data.groupby(key_column)[value_column].agg()
```

In [27]:
# 위에서 만든 df파일을 다시 쓰자.
df

,key1,key2,data1,data2
0,a,one,-0.255730,-1.097117
1,a,two,0.263433,0.265841
2,b,one,0.963786,-0.239807
3,b,two,0.007764,-0.077920
4,a,one,0.859010,-0.051924


In [28]:
# agg() 안에 함수를 직접 집어넣어서 사용할 수 있다.
df.groupby('key1').agg('mean')

,data1,data2
key1,,
a,0.288904,-0.294400
b,0.485775,-0.158864


In [29]:
# 또한, agg()를 사용하면 여러 통계치를 동시에 확인할 수도 있다.
df.groupby('key1')['data1'].agg(['mean', 'std'])

,mean,std
key1,,
a,0.288904,0.557807
b,0.485775,0.676010


In [30]:
# agg() 안에 들어가는 것은 '함수' 이므로, def로 정의된 함수나 lambda를 사용할 수도 있다.
df.groupby('key1').agg(lambda x: np.mean(x))

,data1,data2
key1,,
a,0.288904,-0.294400
b,0.485775,-0.158864


In [31]:
# 여러 lambda를 사용할 수도 있다.
df.groupby('key1').agg([lambda x: np.mean(x), lambda x: np.std(x)])

SpecificationError: Function names must be unique, found multiple named <lambda>

In [32]:
# 하나의 value column에 대해 여러 개의 집계함수를 사용할 경우, 
# (함수이름, 함수) 형태로 집계함수의 이름을 지정할 수 있다.
df.groupby('key1')['data1'].agg([('평균','mean'), ('표준편차','std')])

,평균,표준편차
key1,,
a,0.288904,0.557807
b,0.485775,0.676010


In [33]:
# 여러 value column에 대해 각각 다른 함수를 적용하고 싶은 경우, dict() 형태로 인자를 넘겨준다.
df.groupby('key1').agg({'data1' : 'mean', 'data2' : 'std'})

,data1,data2
key1,,
a,0.288904,0.713099
b,0.485775,0.114471


In [42]:
# 예제. 위에서 사용한 파일을 다시 쓰시오.
#1. 각 학생의 모든 성적에 대하여, 각 학생 성적의 (최댓값 - 최솟값)을 구하시오.

source.groupby(['이름'])['성적'].agg(lambda x: x.max()-x.min()).reset_index()

#2. 각 성별별 성적의 평균과 분산을 동시에 구하시오.

source.groupby(['성별'])['성적'].agg(['mean','var']).reset_index()

,성별,mean,var
0,남,67,223.333333
1,여,79,143.333333


# Pivot table
<br><img align="left" src="http://drive.google.com/uc?export=view&id=1HEBp4qq4GaksdQBb2fx2tRsq-gVR-j6d" width=800 height=600>

- 데이터를 바탕으로, 서로 다른 column의 조합을 표로 만들어서 데이터를 일목요연하게 보기 위한 도구.
- 피벗테이블을 잘 활용하면, 데이터를 상황에 맞춰 내가 원하는 조합으로 결합해 한눈에 볼 수 있다.

- 피벗테이블은 다음과 같은 형식으로 사용한다.
```python
pd.pivot_table(data, values = column_name, index = column_name, 
                 columns = column_name, aggfunc = function)
```

In [43]:
tr = pd.DataFrame({
        'id': [1,1,1,1,2,2,2],
        'site': ['a','b','c','a','a','b','b'],
        'pageview': np.arange(1,8),
        'dwelltime': np.arange(7.0, 0, -1),
    }, columns=['id','site','pageview','dwelltime'])

tr

,id,site,pageview,dwelltime
0,1,a,1,7.0
1,1,b,2,6.0
2,1,c,3,5.0
3,1,a,4,4.0
4,2,a,5,3.0
5,2,b,6,2.0
6,2,b,7,1.0


In [44]:
# 피벗 테이블은 아래와 같이 만들 수 있다.
pd.pivot_table(tr, values='pageview', index='id', columns='site', 
               aggfunc=sum)

site,a,b,c
id,,,
1,5.0,2.0,3.0
2,5.0,13.0,NaN


In [45]:
# 만약 빈 값이 있다면, fill_value 인자를 통해 값을 넣어줄 수 있다.
pd.pivot_table(tr, values='pageview', index='id', columns='site', 
               aggfunc=sum, fill_value=0)

site,a,b,c
id,,,
1,5,2,3
2,5,13,0


In [46]:
# 피벗 테이블의 가장 큰 장점은, 원하는 column끼리의 조합으로 새로운 DataFrame을 만들 수 있다는 점이다.
pd.pivot_table(tr, values='dwelltime', index='site', columns='id', 
               aggfunc=sum, fill_value=0)

id,1,2
site,,
a,11,3
b,6,3
c,5,0


In [47]:
# aggfunc 부분에 다양한 함수를 적용할 수 있다.
pd.pivot_table(tr, values='pageview', index='id', 
               aggfunc=np.mean, fill_value=0).reset_index()

,id,pageview
0,1,2.5
1,2,6.0


In [48]:
# 물론, lambda나 def 함수도 적용할 수 있다.
pd.pivot_table(tr, values='pageview', index='id', 
               aggfunc=lambda x: max(x) - min(x), fill_value=0).reset_index()

,id,pageview
0,1,3
1,2,2
